In [66]:
import numba
import numpy as np

In [62]:
# simple example of using numba
# in this case, the native python code is surprisingly OK!
# the C code below is only slightly faster than the numba version

def f(x,N):
    tmp = 0
    for i in range(N):
        tmp += i**x
    return tmp

x = 1.5
N = 10000000

fast_f = numba.jit(nopython=True)(f)

print(f"{f(x,N)=} {fast_f(x,N)=}")

print("")
print("time for naive python code")
%time f(x,N)

print("")
print("time for JIT-ed code code")
%time fast_f(x,N)

f(x,N)=1.2649109059538203e+17 fast_f(x,N)=1.2649109059538203e+17

time for naive python code
CPU times: user 961 ms, sys: 3.37 ms, total: 965 ms
Wall time: 967 ms

time for JIT-ed code code
CPU times: user 308 ms, sys: 1.41 ms, total: 309 ms
Wall time: 309 ms


1.2649109059538203e+17

In [65]:
# c code equivalent

code = """
#import <math.h>
#import <stdlib.h>
#include <stdio.h>
double f(double x, int N){
    double tmp = 0;
    for(int i=0; i<N; i++){
        tmp += pow(i,x);
    }
    return(tmp);
}
int main(int argc, char *argv[]){
    char* pEnd;
    double x = strtof(argv[1], &pEnd);
    double N = strtof(argv[2], &pEnd);
    printf("%f",f(x,N));
}
"""
f = open("tmp.c","w")
f.write(code)
f.close()
import os
os.system("gcc -O2 tmp.c")
os.system(f"time ./a.out {x} {N};")

126491090595382032.000000


real	0m0.273s
user	0m0.203s
sys	0m0.004s


0

In [93]:
# better example—matrix multiplication

def f(X,Y):
    out = np.zeros((X.shape[0],Y.shape[1]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[1]):
            for k in range(X.shape[1]):
                out[i,j] += X[i,k] * Y[k,j]
    return out

X = np.random.randn(100,150)
Y = np.random.randn(150,200)

fast_f = numba.jit(nopython=True)(f)

assert np.allclose(f(X,Y),fast_f(X,Y))
assert np.allclose(fast_f(X,Y), X @ Y)

print("")
print("time for naive python code")
%time f(X,Y)

print("")
print("time for JIT-ed code code")
%time fast_f(X,Y);


time for naive python code
CPU times: user 3.21 s, sys: 33.5 ms, total: 3.25 s
Wall time: 1.94 s

time for JIT-ed code code
CPU times: user 3.22 ms, sys: 1 µs, total: 3.22 ms
Wall time: 3.24 ms


In [113]:
# now—could we do something interesting using a custom class?

class Array:
    def __init__(self, shape):
        self.data = np.zeros(shape)
    def __getitem__(self,idx):
        return self.data.__getitem__(idx)
    def __setitem__(self,idx,value):
        self.data.__setitem__(idx,value)
    @property
    def shape(self):
        return self.data.shape

class ArrayType(numba.types.Type):
    def __init__(self):
        super(ArrayType, self).__init__(name='Array')

array_type = ArrayType()


@numba.extending.typeof_impl.register(Array)
def typeof_index(val, c):
    return array_type

numba.extending.as_numba_type.register(Array, array_type)

@numba.extending.type_callable(Array)
def type_interval(context):
    def typer(shape):
        if all(isinstance(i,int) for i in shape):
            return array_type
    return typer

def randn(*sizes):
    X = Array(sizes)
    X.data = np.random.randn(*sizes)
    return X

X = randn(10,15)
Y = randn(15,20)

# better example—matrix multiplication

def f(X,Y):
    out = Array((X.shape[0],Y.shape[1]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[1]):
            for k in range(X.shape[1]):
                out[i,j] += X[i,k] * Y[k,j]
    return out

fast_f = numba.jit(nopython=True)(f)

Z = f(X,Y)
Z2 = fast_f(X,Y)
assert np.allclose(Z.data,Z2.data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'shape' of type Array

File "../../../../../var/folders/5z/ngs4vf717m12n9fd_sfbyyyw0000gn/T/ipykernel_5045/498799954.py", line 45:
<source missing, REPL/exec in use?>

During: typing of get attribute at /var/folders/5z/ngs4vf717m12n9fd_sfbyyyw0000gn/T/ipykernel_5045/498799954.py (45)

File "../../../../../var/folders/5z/ngs4vf717m12n9fd_sfbyyyw0000gn/T/ipykernel_5045/498799954.py", line 45:
<source missing, REPL/exec in use?>
